In [19]:
import os
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import classification_report

all_results_repos = {'name': 'Path'}
all_results_repos['anxietyE2E/results_archive'] = '/mnt/d/Users/alkurdi/anxietyE2E/results_archive'
all_results_repos['anxietyE2E/results'] = '/mnt/d/Users/alkurdi/anxietyE2E/results'
all_results_repos['anxietyE2E/results_reduced'] = '/mnt/d/Users/alkurdi/anxietyE2E/results_reduced'
all_results_repos['wsl2 may2023'] = '/mnt/d/Users/alkurdi/from_wsl2_created_may_23/wsl2_e2e/results'
all_results_repos['delta old e2e_snr_0.5'] = '/mnt/d/Users/alkurdi/delta_old_results/e2e/e2e/results_snr_0.5'
all_results_repos['delta old e2e_copy_snr_0.1'] = '/mnt/d/Users/alkurdi/delta_old_results/e2e/e2e_copy/results_snr_0.1'
all_results_repos['delta old e2e_copy_snr_0.2'] = '/mnt/d/Users/alkurdi/delta_old_results/e2e/e2e_copy/results_snr_0.2'
all_results_repos['delta old e2e_copy_snr_0.5'] = '/mnt/d/Users/alkurdi/delta_old_results/e2e/e2e_copy/results_snr_0.5'
all_results_repos['delta old e2e_copy2_snr_0.01'] = '/mnt/d/Users/alkurdi/delta_old_results/e2e/e2e_copy2/results_snr_0.01'
all_results_repos['delta old e2e_copy2_snr_0.1'] = '/mnt/d/Users/alkurdi/delta_old_results/e2e/e2e_copy2/results_snr_0.1'
all_results_repos['delta old e2e_copy2_snr_0.2'] = '/mnt/d/Users/alkurdi/delta_old_results/e2e/e2e_copy2/results_snr_0.2'
all_results_repos['delta old e2e_copy2_snr_0.5'] = '/mnt/d/Users/alkurdi/delta_old_results/e2e/e2e_copy2/results_snr_0.5'
all_results_repos['delta old e2e_snr_0.2_snr_0.2'] = '/mnt/d/Users/alkurdi/delta_old_results/e2e/e2e_snr_0.2/results_snr_0.2'
all_results_repos['delta old e2e_snr_0.2_snr_0.5'] = '/mnt/d/Users/alkurdi/delta_old_results/e2e/e2e_snr_0.2/results_snr_0.5'
all_results_repos['bbyn katerina snr_0.5'] = '/mnt/d/Users/alkurdi/delta_old_results/e2e_bbyn/katerina/results_snr_0.5'
all_results_repos['bbyn katerina snr_0.5_save'] = '/mnt/d/Users/alkurdi/delta_old_results/e2e_bbyn/katerina/results_snr_0.5_save'


import warnings
warnings.filterwarnings("ignore")
done_list = {}
master_ledger = {'name':['df_best_model', 'df_metrics', 'history', 'predictions', 'result', 'aggregated_classification_reports', 'done']}
if os.path.isfile('/mnt/d/Users/alkurdi/master_ledger.pkl'):
    with open('/mnt/d/Users/alkurdi/master_ledger.pkl', 'rb') as handle:
        master_ledger = pickle.load(handle)
else:
        
    for name, Path in all_results_repos.items():
        if name != 'name':        
            fold_gen = (i for i in os.listdir(Path) if 'WESAD' in i)
            for fold in fold_gen:
                tune_gen = (i for i in os.listdir(Path+'/'+fold) if 'tune' in i)
                for tune in tune_gen:
                    model_gen = (i for i in os.listdir(Path+'/'+fold+'/'+str(tune)))
                    for model in model_gen:
                        it_gen = ( i for i in os.listdir(Path+'/'+fold+'/'+str(tune)+'/'+model) if 'it' in i)
                        for it in it_gen:
                            try: 
                                in_path = Path+'/'+fold+'/'+str(tune)+'/'+model+'/'+str(it)
                                df_best_model = pd.read_csv(in_path+'/df_best_model.csv')
                                df_metrics = pd.read_csv(in_path+'/df_metrics.csv')
                                history = pd.read_csv(in_path+'/history.csv')
                                aggregated_classification_reports = {}
                                metrics = ["precision", "recall", "f1-score", "support"]
                                with open(in_path+'/predictions.txt') as f:
                                    y_true = [int(x) for x in f.readline().split()]
                                    if len(aggregated_classification_reports) == 0:
                                        for clas in set(y_true):
                                            aggregated_classification_reports[clas] = {"f1-score": [], "precision": [], "recall": [],
                                                                                        "support": []}
                                    y_pred = [int(x) for x in f.readline().split()]
                                    report = classification_report(y_true, y_pred, output_dict=True)
                                    for clas in aggregated_classification_reports:
                                        for metric in metrics:
                                            aggregated_classification_reports[clas][metric].append(report[str(clas)][metric])
                                result = []
                                for clas in aggregated_classification_reports:
                                    row = ['WESAD', clas]
                                    for metric in metrics:
                                        metric_values = aggregated_classification_reports[clas][metric]
                                        row.append(np.mean(metric_values))
                                        row.append(np.std(metric_values))
                                    result.append(row[:-1])  # Remove support std
                                is_done = os.path.isdir(in_path+'/DONE')
                                #
                                if is_done:
                                    done_list[in_path] = is_done
                                master_ledger[name+'_'+fold+'_'+tune+'_'+model+'_'+it] = [df_best_model, df_metrics, history, result, aggregated_classification_reports, is_done]
                            except Exception as e:
                                master_ledger[name+'_'+fold+'_'+tune+'_'+model+'_'+it] = [None, None, None, None, None, None]
                                #print('during '+name+'_'+fold+'_'+tune+'_'+model+'_'+it+' an error occured: ')
                                #print(e)
                                continue                    

    with open('/mnt/d/Users/alkurdi/master_ledger.pkl', 'wb') as handle:
        pickle.dump(master_ledger, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [20]:
len(master_ledger)

4878

In [3]:
'''
i = 0
imax = len(master_ledger)
for key in master_ledger:
    i += 1
    if i % 50 == 0 :
        print(i, '/', imax, '',round(i/imax*100,2), '% done')
    
master_ledger_df.to_pickle('/mnt/d/Users/alkurdi/master_ledger_df.pkl')
'''

"\ni = 0\nimax = len(master_ledger)\nfor key in master_ledger:\n    i += 1\n    if i % 50 == 0 :\n        print(i, '/', imax, '',round(i/imax*100,2), '% done')\n    \nmaster_ledger_df.to_pickle('/mnt/d/Users/alkurdi/master_ledger_df.pkl')\n"

In [5]:
#create a multiprocess function to run the following code in parallel

def process_each_ledger(key):
    ledger_df = pd.DataFrame()
    if key != 'name':  
        case_df = pd.DataFrame()
        if master_ledger[key][-2] is None:
            pass
        else: 
            case_df['f1 binary']= master_ledger[key][-2][0]['f1-score']
            pass
        for ind in master_ledger[key]:
            pass
        print('\n',key.split('_'),'\n')
        case_df['it'] = key.split('_')[-1]
        case_df['model'] = key.split('_')[-3]
        case_df['tune'] = key.split('_')[-4]
        case_df['fold'] = key.split('_')[-6]
        case_df['name'] = key
        ledger_df = pd.concat([ledger_df, master_ledger[key][0], master_ledger[key][1], case_df], axis = 1)
    return ledger_df


In [16]:

import multiprocessing as mp
from multiprocessing import Pool
import time
from tqdm import tqdm


start = time.time()
len(master_ledger)
with Pool() as pool:
    ledger_results = pool.map(process_each_ledger, master_ledger.keys())
    #ledger_results = tqdm(pool.imap(process_each_ledger, list(master_ledger.keys())), total=len(master_ledger))
    print('done')

end = time.time()
print(end - start)


done
2.2542431354522705


In [18]:
import concurrent.futures
import logging
logging.basicConfig(filename='logs/analyze_all_case_results.log',level=logging.INFO)

start_time = time.time()

with concurrent.futures.ProcessPoolExecutor(max_workers=8) as executor:
        futures = [executor.submit(process_each_ledger, i) for i in master_ledger.keys()]
        done, not_done = concurrent.futures.wait(futures, return_when='ALL_COMPLETED')
        logging.debug(f'done assigning futures, now waiting for them to finish')
        for future in done:
            ledger_results = future.result()
        
imax =len(master_ledger)
logging.info(f'finished {imax} iterations in {round(time.time()-start_time,2)} seconds')
logging.info(f'an average of {(time.time()-start_time)/imax} seconds per iteration')



with open('/mnt/d/Users/alkurdi/ledger_results.pkl', 'wb') as f:
            pickle.dump(ledger_results, f)
            


False

In [34]:
with open('/mnt/d/Users/alkurdi/ledger_results.pkl', 'rb') as handle:
        ledger_results= pickle.load( handle)
#master_ledger_df = pd.DataFrame()
#for i in ledger_results:
#    master_ledger_df = pd.concat([master_ledger_df, i])
#display(master_ledger_df)

,accuracy,auc,best_model_learning_rate,best_model_nb_epoch,best_model_train_acc,best_model_train_loss,best_model_val_acc,best_model_val_loss,duration,f1,f1 binary,fold,it,model,name,precision,recall,tune
0,0.529680,0.500000,0.0,0.0,0.528443,7.558744,0.532110,7.541494,595.331101,0.230846,0.692537,03,04,encoderM,anxietyE2E/results_archive_WESAD_5fold_03_tune...,0.176560,0.333333,01
0,0.295455,0.593788,0.0,2.0,0.298193,1.376488,0.312217,1.138777,127.240494,0.152047,0.000000,00,00,fcnM,delta old e2e_copy_snr_0.2_WESAD_5fold_00_tune...,0.098485,0.333333,00
0,0.650000,0.688982,0.0,3.0,0.638554,0.838200,0.714932,0.878387,92.111711,0.477549,0.755725,00,01,fcnM,delta old e2e_snr_0.2_snr_0.2_WESAD_5fold_00_t...,0.453227,0.507692,06
0,0.777273,0.845686,0.0,104.0,0.744713,0.575286,0.766816,0.475050,718.436639,0.578112,0.854902,02,00,fcnM,delta old e2e_copy_snr_0.2_WESAD_5fold_02_tune...,0.537137,0.625858,09
0,0.671233,0.748436,0.0,149.0,0.772455,0.712756,0.770642,0.708124,43.993620,0.472861,0.758389,03,02,cnnM,bbyn katerina snr_0.5_save_WESAD_5fold_03_tune...,0.513266,0.496430,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.804545,0.843354,0.0,54.0,0.741692,0.610437,0.753363,0.491319,398.491518,0.588280,0.883721,02,02,fcnM,delta old e2e_copy2_snr_0.2_WESAD_5fold_02_tun...,0.540659,0.645111,09
0,0.663717,0.699577,0.0,146.0,0.800303,0.521738,0.697248,0.718021,5199.148379,0.518739,0.744186,04,02,resnetM,bbyn katerina snr_0.5_WESAD_5fold_04_tune_05_r...,0.518894,0.524708,05
0,0.813636,0.818692,0.0,19.0,0.948795,0.224816,0.710407,0.614418,24.573870,0.679242,0.862595,00,04,cnnM,anxietyE2E/results_archive_WESAD_5fold_00_tune...,0.754088,0.682261,05
0,0.740909,0.771898,0.0,16.0,0.930723,0.283418,0.714932,0.609809,21.913184,0.577197,0.829630,00,02,cnnM,anxietyE2E/results_archive_WESAD_5fold_00_tune...,0.597760,0.587914,05


In [48]:
master_ledger_df2 = pd.concat(ledger_results)

In [49]:
master_ledger_df2

,accuracy,auc,best_model_learning_rate,best_model_nb_epoch,best_model_train_acc,best_model_train_loss,best_model_val_acc,best_model_val_loss,duration,f1,f1 binary,fold,it,model,name,precision,recall,tune
0,0.529680,0.500000,0.0,0.0,0.528443,7.558744,0.532110,7.541494,595.331101,0.230846,0.692537,03,04,encoderM,anxietyE2E/results_archive_WESAD_5fold_03_tune...,0.176560,0.333333,01
0,0.295455,0.593788,0.0,2.0,0.298193,1.376488,0.312217,1.138777,127.240494,0.152047,0.000000,00,00,fcnM,delta old e2e_copy_snr_0.2_WESAD_5fold_00_tune...,0.098485,0.333333,00
0,0.650000,0.688982,0.0,3.0,0.638554,0.838200,0.714932,0.878387,92.111711,0.477549,0.755725,00,01,fcnM,delta old e2e_snr_0.2_snr_0.2_WESAD_5fold_00_t...,0.453227,0.507692,06
0,0.777273,0.845686,0.0,104.0,0.744713,0.575286,0.766816,0.475050,718.436639,0.578112,0.854902,02,00,fcnM,delta old e2e_copy_snr_0.2_WESAD_5fold_02_tune...,0.537137,0.625858,09
0,0.671233,0.748436,0.0,149.0,0.772455,0.712756,0.770642,0.708124,43.993620,0.472861,0.758389,03,02,cnnM,bbyn katerina snr_0.5_save_WESAD_5fold_03_tune...,0.513266,0.496430,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.804545,0.843354,0.0,54.0,0.741692,0.610437,0.753363,0.491319,398.491518,0.588280,0.883721,02,02,fcnM,delta old e2e_copy2_snr_0.2_WESAD_5fold_02_tun...,0.540659,0.645111,09
0,0.663717,0.699577,0.0,146.0,0.800303,0.521738,0.697248,0.718021,5199.148379,0.518739,0.744186,04,02,resnetM,bbyn katerina snr_0.5_WESAD_5fold_04_tune_05_r...,0.518894,0.524708,05
0,0.813636,0.818692,0.0,19.0,0.948795,0.224816,0.710407,0.614418,24.573870,0.679242,0.862595,00,04,cnnM,anxietyE2E/results_archive_WESAD_5fold_00_tune...,0.754088,0.682261,05
0,0.740909,0.771898,0.0,16.0,0.930723,0.283418,0.714932,0.609809,21.913184,0.577197,0.829630,00,02,cnnM,anxietyE2E/results_archive_WESAD_5fold_00_tune...,0.597760,0.587914,05
